In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import gzip
import json
from pymongo import MongoClient
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
purchasing_df = getDF('Data/Video_Games_5.json.gz')
meta_datadf = getDF('Data/meta_Video_Games.json.gz')

In [ ]:
columns_to_drop = ['vote', 'style', 'image'] 
purchasing_df = purchasing_df.drop(columns=columns_to_drop)
positive_samples = purchasing_df.copy()
positive_samples['label'] = 1 
positive_samples['reviewTime'] = pd.to_datetime(positive_samples['reviewTime'])
positive_samples.head()

In [ ]:
meta_datadf = meta_datadf.drop_duplicates(subset='asin', keep='first')
meta_datadf.reset_index(drop=True, inplace=True)
meta_datadf.head()

In [ ]:
positive_samples = positive_samples.merge(meta_datadf, on='asin', how='inner')
positive_samples['price'] = pd.to_numeric(positive_samples['price'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
positive_samples['price'].isna().sum()

In [ ]:
users_to_instances = {} 
for index, row in positive_samples.iterrows(): 
    if row['reviewerID'] not in users_to_instances:
        users_to_instances[row['reviewerID']] = 0
    users_to_instances[row['reviewerID']] += 1

In [ ]:
positive_samples = positive_samples[positive_samples['reviewerID'].map(users_to_instances) >= 10]

In [ ]:
item_to_instances = {} 
for index, row in positive_samples.iterrows(): 
    if row['asin'] not in item_to_instances:
        item_to_instances[row['asin']] = 0
    item_to_instances[row['asin']] += 1

In [ ]:
positive_samples = positive_samples[positive_samples['asin'].map(item_to_instances) >= 10]

In [ ]:
positive_samples.shape

In [ ]:
columns_to_drop = ['imageURL', 'imageURLHighRes', 'reviewerName', 'reviewText', 
                   'unixReviewTime', 'date']
positive_samples = positive_samples.drop(columns=columns_to_drop, axis=1)
positive_samples = positive_samples.reset_index(drop=True)

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client['video_games_db']
price_collection = db['keepa']
query = {"df_NEW": {"$exists": True}}
document_count = price_collection.count_documents(query)
projection = {"df_NEW": 1, "asin": 1, "_id": 0}
documents = price_collection.find(query, projection)
price_df = pd.DataFrame(documents) 

In [ ]:
items_considered = set() 
for index, row in positive_samples.iterrows(): 
    items_considered.add(row["asin"])

In [ ]:
asin_to_price = {}
for index, row in price_df.iterrows():
    asin_to_price[row["asin"]] = row["df_NEW"]

In [ ]:
asin_to_positive = {} 
for index, row in positive_samples.iterrows(): 
    asin_to_positive[row['asin']] = index

In [ ]:
asin_to_meta = {}
for index, row in meta_datadf.iterrows(): 
    asin_to_meta[row["asin"]] = index

In [ ]:
person_to_purchases = {} 
for index, row in positive_samples.iterrows(): 
    if row['reviewerID'] not in person_to_purchases:
        person_to_purchases[row['reviewerID']] = set()
    person_to_purchases[row['reviewerID']].add(row['asin'])

In [ ]:
person_to_not_purchased = {} 
all_asins = set()
for index, row in positive_samples.iterrows(): 
    all_asins.add(row['asin'])

In [ ]:
for index, row in positive_samples.iterrows():
    if row['reviewerID'] not in person_to_not_purchased:
        person_to_not_purchased[row['reviewerID']] = all_asins.copy()
    if row['asin'] in person_to_not_purchased[row['reviewerID']]:
        person_to_not_purchased[row['reviewerID']].remove(row['asin'])

In [ ]:
def binary_search_closest(arr, target, compare):
    low, high = 0, len(arr) - 1
    closest = None
    closest_diff = float('inf')
    while low <= high:
        mid = (low + high) // 2
        result = compare(arr[mid], target)
        if result == 0:
            return arr[mid]
        elif result < 0:
            low = mid + 1
        else:
            high = mid - 1
        diff = abs(result)
        if diff < closest_diff:
            closest_diff = diff
            closest = arr[mid]
    return closest

In [ ]:
from tqdm import tqdm
tqdm.pandas()
removed_rows = set()
for index, row in tqdm(positive_samples.iterrows(), total=positive_samples.shape[0], desc="Processing rows"):
    asin = row["asin"]
    current_date = row["reviewTime"]
    if asin in asin_to_price:
        keys_to_be_removed = []
        for key in asin_to_price[asin]:
            if np.isnan(asin_to_price[asin][key]):
                keys_to_be_removed.append(key)
        for key in keys_to_be_removed:
            del asin_to_price[asin][key]
        price_list = list(asin_to_price[asin].keys())
        if len(price_list) == 0:
            removed_rows.add(index)
        else: 
            closest_date = binary_search_closest(price_list, current_date, lambda x, y: (pd.to_datetime(x) - y).total_seconds())
            closest_price = asin_to_price[asin][closest_date]
            positive_samples.at[index, "price"] = closest_price
    else:
        removed_rows.add(index)
print(len(removed_rows))                                                                                                                                                        

In [ ]:
positive_samples = positive_samples.drop(removed_rows)
positive_samples.reset_index(drop=True, inplace=True)

In [ ]:
print(positive_samples.shape)

In [ ]:
positive_samples = positive_samples.dropna(subset=['price'])
positive_samples.reset_index(drop=True, inplace=True)

In [ ]:
print(positive_samples.shape)

In [ ]:
positive_samples = positive_samples.sample(frac=1).reset_index(drop=True)

In [8]:
positive_samples = pd.read_csv('../samples/positive_samples.csv')

/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_80809/1724587496.py:1: DtypeWarning: Columns (13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  positive_samples = pd.read_csv('../samples/positive_samples.csv')


In [9]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
brand_encoder = LabelEncoder()
positive_samples['user_id_encoded'] = user_encoder.fit_transform(positive_samples['reviewerID'])
positive_samples['item_id_encoded'] = item_encoder.fit_transform(positive_samples['asin'])
positive_samples['brand_id_encoded'] = brand_encoder.fit_transform(positive_samples['brand'])
positive_samples = positive_samples[['user_id_encoded', 'item_id_encoded', 'brand_id_encoded', 'price', 'description', 'label']]
num_users =  positive_samples["user_id_encoded"].nunique()
num_items = positive_samples["item_id_encoded"].nunique()
num_brands =  positive_samples["brand_id_encoded"].nunique()

In [10]:
X = positive_samples[['user_id_encoded', 'item_id_encoded', 'brand_id_encoded', 'price', 'description']].values
y = positive_samples['label'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

positive_indices = np.where(y_train == 1)[0]
X_train_positive = X_train[positive_indices]
y_train_positive = y_train[positive_indices]
positive_indices_test = np.where(y_test == 1)[0]
X_test_positive = X_test[positive_indices_test]
y_test_positive = y_test[positive_indices_test]

X_train_user = X_train[:, 0].astype(np.int32)
X_train_item = X_train[:,1].astype(np.int32)
X_train_brand = X_train[:,2].astype(np.int32)
X_train_price = X_train[:,3].astype(np.float32)
X_train_description = X_train[:, 4].astype(np.str_)

X_test_user = X_test[:, 0].astype(np.int32)
X_test_item = X_test[:, 1].astype(np.int32)
X_test_brand = X_test[:, 2].astype(np.int32)
X_test_price = X_test[:, 3].astype(np.float32)
X_test_description = X_test[:, 4].astype(np.str_)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [12]:
tokenizer = get_tokenizer("basic_english")
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)
vocab = build_vocab_from_iterator(yield_tokens(X_train_description), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

X_train_description_processed = [text_pipeline(text) for text in X_train_description]
X_test_description_processed = [text_pipeline(text) for text in X_test_description]


In [13]:
max_length = max(len(desc) for desc in X_train_description_processed + X_test_description_processed)

def pad_sequence(seq):
    return seq + [vocab["<unk>"]] * (max_length - len(seq))

X_train_description_padded = torch.tensor([pad_sequence(desc) for desc in X_train_description_processed], dtype=torch.long)
X_test_description_padded = torch.tensor([pad_sequence(desc) for desc in X_test_description_processed], dtype=torch.long)

In [14]:
class UserTower(nn.Module):
    def __init__(self, user_dim, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(user_dim, embedding_dim)
        self.fc = nn.Linear(embedding_dim, 64)
        
    def forward(self, x):
        x = self.embedding(x)
        return self.fc(x)

In [15]:
class ItemTower(nn.Module):
    def __init__(self, item_dim, brand_dim, vocab_size, embedding_dim, description_embedding_size):
        super().__init__()
        self.item_embedding = nn.Embedding(item_dim, embedding_dim)
        self.brand_embedding = nn.Embedding(brand_dim, embedding_dim)
        self.description_embedding = nn.Embedding(vocab_size, description_embedding_size)
        self.fc = nn.Linear(embedding_dim * 2 + 1 + description_embedding_size, 64)
        
    def forward(self, item, brand, price, description):
        item_emb = self.item_embedding(item)
        brand_emb = self.brand_embedding(brand)
        desc_vec = self.description_embedding(description).mean(dim=1)
        x = torch.cat([item_emb, brand_emb, desc_vec, price.unsqueeze(1)], dim=1)
        return self.fc(x)

In [16]:
class TwoTowerModel(nn.Module):
    def __init__(self, user_dim, item_dim, brand_dim, vocab_size, embedding_dim, description_embedding_size):
        super().__init__()
        self.user_tower = UserTower(user_dim, embedding_dim)
        self.item_tower = ItemTower(item_dim, brand_dim, vocab_size=vocab_size, embedding_dim = embedding_dim,description_embedding_size=description_embedding_size)
        
    def forward(self, user, item, brand, price, description):
        user_emb = self.user_tower(user)
        item_emb = self.item_tower(item, brand, price, description)
        return (user_emb * item_emb).sum(dim=1)

In [17]:
def in_batch_negative_sampling(user, item, brand, price, description):
    batch_size = user.size(0)
    users = user.repeat_interleave(batch_size)
    items = item.repeat(batch_size)
    brands = brand.repeat(batch_size)
    prices = price.repeat(batch_size)
    descriptions = description.repeat(batch_size, 1)
    labels = torch.eye(batch_size).view(-1)
    positive_indices = [i * (batch_size + 1) for i in range(batch_size)]
    pos_users = users[positive_indices]
    pos_items = items[positive_indices]
    pos_brands = brands[positive_indices]
    pos_prices = prices[positive_indices]
    pos_description = descriptions[positive_indices]
    pos_labels = labels[positive_indices]
    neg_indices = []
    for i in range(batch_size):
        start = i * batch_size
        end = (i + 1) * batch_size
        possible_neg = list(range(start, end))
        possible_neg.remove(start + i) 
        neg_indices.extend(random.sample(possible_neg, 1))
    users = users[neg_indices]
    items = items[neg_indices]
    brands = brands[neg_indices]
    prices = prices[neg_indices]
    descriptions = descriptions[neg_indices]
    labels = labels[neg_indices]
    users = torch.cat([pos_users, users])
    items = torch.cat([pos_items, items])
    brands = torch.cat([pos_brands, brands])
    prices = torch.cat([pos_prices, prices])
    descriptions = torch.cat([pos_description, descriptions])
    labels = torch.cat([pos_labels, labels])
    return users, items, brands, prices, descriptions, labels

In [20]:
def calculate_accuracy(outputs, labels):
    predictions = (torch.sigmoid(outputs) > 0.5).float()
    correct = (predictions == labels).float().sum()
    return correct / len(labels)

def train_model(model, train_loader, test_loader, optimizer, epochs):
    criterion = nn.BCEWithLogitsLoss()
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_accuracy = 0
        num_batches = 0
        all_outputs = []
        all_labels = []
        for batch in train_loader:
            user, item, brand, price, description, labels = batch
            #print the shapes
            users, items, brands, prices, description, labels = in_batch_negative_sampling(user, item, brand, price, description)
            optimizer.zero_grad()
            outputs = model(users, items, brands, prices, description)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(outputs, labels)
            num_batches += 1
            all_outputs.extend(outputs.detach().cpu().numpy())
            all_labels.extend(labels.detach().cpu().numpy())
            
        train_auc = roc_auc_score(all_labels, all_outputs)
        train_loss = total_loss / num_batches
        train_accuracy = total_accuracy / num_batches

        model.eval()
        total_val_loss = 0
        total_val_accuracy = 0
        num_val_batches = 0
        all_val_outputs = []
        all_val_labels = []
        
        with torch.no_grad():
            for batch in test_loader:
                user, item, brand, price, description, _ = batch
                users, items, brands, prices, description, labels = in_batch_negative_sampling(user, item, brand, price, description)
                outputs = model(users, items, brands, prices, description)
                val_loss = criterion(outputs, labels)
                total_val_loss += val_loss.item()
                total_val_accuracy += calculate_accuracy(outputs, labels)
                num_val_batches += 1
                all_val_outputs.extend(outputs.detach().cpu().numpy())
                all_val_labels.extend(labels.detach().cpu().numpy())
        
        val_loss = total_val_loss / num_val_batches
        val_accuracy = total_val_accuracy / num_val_batches
        
        model.train()
        val_auc = roc_auc_score(all_val_labels, all_val_outputs)
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Train AUC: {train_auc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        print(f"Val AUC: {val_auc:.4f}")
        print("--------------------")


In [21]:
X_train_user = torch.tensor(X_train_user, dtype=torch.int32)
X_train_item = torch.tensor(X_train_item, dtype=torch.int32)
X_train_brand = torch.tensor(X_train_brand, dtype=torch.int32)
X_train_price = torch.tensor(X_train_price, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_test_user = torch.tensor(X_test_user, dtype=torch.int32)
X_test_item = torch.tensor(X_test_item, dtype=torch.int32)
X_test_brand = torch.tensor(X_test_brand, dtype=torch.int32)
X_test_price = torch.tensor(X_test_price, dtype=torch.float32)

y_test = torch.tensor(y_test, dtype=torch.float32)


train_dataset = TensorDataset(X_train_user, X_train_item, X_train_brand, X_train_price, X_train_description_padded, y_train)
test_dataset = TensorDataset(X_test_user, X_test_item, X_test_brand, X_test_price, X_test_description_padded, y_test)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

vocab_size = len(vocab)
model = TwoTowerModel(user_dim=num_users, item_dim=num_items, brand_dim=num_brands, vocab_size=vocab_size, embedding_dim=32, description_embedding_size=32)
optimizer = optim.Adam(model.parameters(), lr=0.01)

train_model(model, train_loader, test_loader, optimizer, epochs=50)


/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_80809/1889991333.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_user = torch.tensor(X_train_user, dtype=torch.int32)
/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_80809/1889991333.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_item = torch.tensor(X_train_item, dtype=torch.int32)
/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_80809/1889991333.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_brand = torch.tensor(X_train_b

Epoch 1/50
Train Loss: 0.9055, Train Accuracy: 0.5017
Train AUC: 0.5019
Val Loss: 0.7196, Val Accuracy: 0.5018
Val AUC: 0.5020
--------------------
Epoch 2/50
Train Loss: 0.7052, Train Accuracy: 0.5129
Train AUC: 0.5184
Val Loss: 0.7037, Val Accuracy: 0.5065
Val AUC: 0.5098
--------------------
Epoch 3/50
Train Loss: 0.7003, Train Accuracy: 0.5253
Train AUC: 0.5361
Val Loss: 0.7078, Val Accuracy: 0.5171
Val AUC: 0.5257
--------------------
Epoch 4/50
Train Loss: 0.7025, Train Accuracy: 0.5496
Train AUC: 0.5717
Val Loss: 0.7074, Val Accuracy: 0.5443
Val AUC: 0.5644
--------------------
Epoch 5/50
Train Loss: 0.6872, Train Accuracy: 0.5869
Train AUC: 0.6231
Val Loss: 0.6988, Val Accuracy: 0.5710
Val AUC: 0.6049
--------------------
Epoch 6/50
Train Loss: 0.6693, Train Accuracy: 0.6244
Train AUC: 0.6740
Val Loss: 0.7040, Val Accuracy: 0.5959
Val AUC: 0.6375
--------------------
Epoch 7/50
Train Loss: 0.6265, Train Accuracy: 0.6711
Train AUC: 0.7311
Val Loss: 0.7147, Val Accuracy: 0.6247
V